In [1]:
!git clone https://github.com/DorBernsohn/Booking-Challenge.git

Cloning into 'Booking-Challenge'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 38 (delta 15), reused 29 (delta 9), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [2]:
import sys
sys.path.append('/content/Booking-Challenge')

# train_path = Path(r"/content/Booking-Challenge/data/train_set.csv")
# test_path = Path(r"/content/Booking-Challenge/data/test_set.csv")

In [3]:
# external
import numpy as np
import pandas as pd

# internal
from utils import (load_data,
                   extract_features,
                   build_time_features,
                   build_prev_city,
                   build_first_city,
                   split_features_label,
                   flatten_features,
                   LabelEncoderMapping)
from config import train_path, test_path

In [4]:
df_train = load_data(train_path, min_trip_length_threshold=4)
df_test = load_data(test_path, min_trip_length_threshold=4)
data = pd.concat([df_train, df_test], sort=False)

In [5]:
df = data.copy()
df = extract_features(df)
features, labels = split_features_label(df)

/content/Booking-Challenge/utils.py:89: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[f'{col}_woy'] = df[col].dt.weekofyear


In [6]:
print(features.shape)
features = features[features.utrip_id.isin(labels[labels.city_id != 0].utrip_id.values)]
labels = labels[labels.city_id != 0]
print(features.shape)

(1255794, 23)
(947789, 23)


In [7]:
df1 = features.set_index(['utrip_id', features.groupby('utrip_id').cumcount()])

mux = pd.MultiIndex.from_product(df1.index.levels, names=df1.index.names)
df2 = df1.reindex(mux, fill_value=1).reset_index(level=1, drop=True).reset_index()
embeded_features = df2.groupby(['utrip_id']).agg(list).reset_index()

In [8]:
embeded_features.head()

,utrip_id,user_id,city_id,length,city_id_encode,device_class_encode,booker_country_encode,hotel_country_encode,affiliate_id_encode,checkin_year_encode,checkin_month_encode,checkin_woy_encode,checkin_dow_encode,checkin_weekend_encode,checkout_year_encode,checkout_month_encode,checkout_woy_encode,checkout_dow_encode,checkout_weekend_encode,trip_length_encode,season_encode,count_encode,first_city
0,1000027_1,"[1000027.0, 1000027.0, 1000027.0, 1.0, 1.0, 1....","[8183.0, 60902.0, 15626.0, 1.0, 1.0, 1.0, 1.0,...","[0.0, 0.6931471805599453, 0.6931471805599453, ...","[4876, 35993, 9205, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[60, 60, 60, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2424, 2424, 2424, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[31, 32, 31, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[31, 32, 32, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[6, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4876, 4876, 4876, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1000033_1,"[1000033.0, 1000033.0, 1000033.0, 1000033.0, 1...","[21328.0, 27485.0, 38677.0, 52089.0, 1.0, 1.0,...","[0.6931471805599453, 0.6931471805599453, 0.693...","[12592, 16275, 22861, 30800, 1, 1, 1, 1, 1, 1,...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[36, 36, 36, 36, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[149, 149, 139, 149, 1, 1, 1, 1, 1, 1, 1, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[14, 14, 13, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[1, 3, 5, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[14, 14, 14, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[3, 5, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 3, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[22861, 22861, 22861, 22861, 1, 1, 1, 1, 1, 1,..."
2,1000045_1,"[1000045.0, 1000045.0, 1000045.0, 1000045.0, 1...","[36170.0, 64876.0, 55128.0, 9608.0, 31817.0, 5...","[0.0, 0.6931471805599453, 0.6931471805599453, ...","[21373, 38323, 32603, 5714, 18789, 34389, 1, 1...","[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[32, 52, 52, 52, 32, 60, 1, 1, 1, 1, 1, 1, 1, ...","[139, 937, 2675, 2675, 2675, 139, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[24, 23, 24, 24, 24, 24, 1, 1, 1, 1, 1, 1, 1, ...","[5, 5, 0, 2, 4, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[24, 24, 24, 24, 24, 25, 1, 1, 1, 1, 1, 1, 1, ...","[6, 0, 2, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 0, 1, 2, 3, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[38323, 38323, 38323, 38323, 38323, 38323, 1, ..."
3,1000083_1,"[1000083.0, 1000083.0, 1000083.0, 1.0, 1.0, 1....","[55990.0, 35160.0, 14705.0, 1.0, 1.0, 1.0, 1.0...","[0.0, 0.0, 0.0, 1.

In [9]:
import tensorflow as tf
dict_slices = tf.data.Dataset.from_tensor_slices((embeded_features.loc[:, ~embeded_features.columns.isin(['utrip_id', 'user_id', 'city_id', 'length'])].to_dict('list'), \
                                                  labels['city_id_encode'].values)).batch(16)

In [10]:
keys = embeded_features.loc[:, ~embeded_features.columns.isin(['utrip_id', 'user_id', 'city_id', 'length'])].keys()

emb_map = {}
for key in keys:
  len_set_feature = max(set([st for row in embeded_features[key] for st in row])) + 1
  emb_map[key]  = (len_set_feature, [int(len_set_feature/4) + 4 if len_set_feature > 4 else len_set_feature][0])
emb_map

{'affiliate_id_encode': (3601, 904),
 'booker_country_encode': (5, 5),
 'checkin_dow_encode': (7, 5),
 'checkin_month_encode': (12, 7),
 'checkin_weekend_encode': (2, 2),
 'checkin_woy_encode': (53, 17),
 'checkin_year_encode': (3, 3),
 'checkout_dow_encode': (7, 5),
 'checkout_month_encode': (12, 7),
 'checkout_weekend_encode': (2, 2),
 'checkout_woy_encode': (53, 17),
 'checkout_year_encode': (2, 2),
 'city_id_encode': (39882, 9974),
 'count_encode': (47, 15),
 'device_class_encode': (3, 3),
 'first_city': (39882, 9974),
 'hotel_country_encode': (196, 53),
 'season_encode': (4, 4),
 'trip_length_encode': (39, 13)}

In [17]:
inputs = {key: tf.keras.layers.Input(shape=(47), name=key) for key in embeded_features.loc[:, ~embeded_features.columns.isin(['utrip_id', 'user_id', 'city_id', 'length'])].keys()}

embeddings = []
for a, key in enumerate(inputs):
    emb = tf.keras.layers.Embedding(emb_map[key][0], 20, name=f'embedding_{key}')(inputs[key])
    embeddings.append(emb)
h = tf.keras.layers.Concatenate()(embeddings)
h = tf.keras.layers.Flatten()(h)
h = tf.keras.layers.Dense(1028, activation='relu')(h)
h = tf.keras.layers.Dropout(0.2)(h)
h = tf.keras.layers.Dense(2048, activation='relu')(h)
h = tf.keras.layers.Dense(4096, activation='relu')(h)
h = tf.keras.layers.Dense(max(labels['city_id_encode'].values) + 1, activation='softmax')(h) # len(list(set([st for row in embeded_features['city_id'] for st in row])))

model_func = tf.keras.Model(inputs=inputs, outputs=h)
model_func.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
model_func.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
city_id_encode (InputLayer)     [(None, 47)]         0                                            
__________________________________________________________________________________________________
device_class_encode (InputLayer [(None, 47)]         0                                            
__________________________________________________________________________________________________
booker_country_encode (InputLay [(None, 47)]         0                                            
__________________________________________________________________________________________________
hotel_country_encode (InputLaye [(None, 47)]         0                                            
____________________________________________________________________________________________

In [12]:
for row, label in dict_slices.take(1):
  print(row)
  print('-'*4)
  print(label)

{'city_id_encode': <tf.Tensor: shape=(16, 47), dtype=int32, numpy=
array([[ 4876, 35993,  9205,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [12592, 16275, 22861, 30800,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1],
       [21373, 38323, 32603,  5714, 18789, 34389,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,

In [13]:
labeled_all_length = [i for i,_ in enumerate(dict_slices)][-1] + 1

train_size = int(0.8 * labeled_all_length)
val_test_size = int(0.1 * labeled_all_length)

df_train = dict_slices.take(train_size)
df_test = dict_slices.skip(train_size)
df_val = df_test.skip(val_test_size)
df_test = df_test.take(val_test_size)

In [18]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model_func.fit(df_train, validation_data=df_val, epochs=20, callbacks=[callback])

Epoch 1/20
10840/10840 [==============================] - 367s 34ms/step - loss: 6.3196 - sparse_top_k_categorical_accuracy: 0.2554 - val_loss: 5.4987 - val_sparse_top_k_categorical_accuracy: 0.3440
Epoch 2/20
10840/10840 [==============================] - 366s 34ms/step - loss: 5.0065 - sparse_top_k_categorical_accuracy: 0.3533 - val_loss: 5.3690 - val_sparse_top_k_categorical_accuracy: 0.3611
Epoch 3/20
10840/10840 [==============================] - 366s 34ms/step - loss: 4.6615 - sparse_top_k_categorical_accuracy: 0.3749 - val_loss: 5.2657 - val_sparse_top_k_categorical_accuracy: 0.3855
Epoch 4/20
10840/10840 [==============================] - 365s 34ms/step - loss: 4.4813 - sparse_top_k_categorical_accuracy: 0.3848 - val_loss: 5.3629 - val_sparse_top_k_categorical_accuracy: 0.3879
Epoch 5/20
10840/10840 [==============================] - 365s 34ms/step - loss: 4.3646 - sparse_top_k_categorical_accuracy: 0.3929 - val_loss: 5.4711 - val_sparse_top_k_categorical_accuracy: 0.3861
Epoch

In [20]:
model_func.evaluate(df_test)

1355/1355 [==============================] - 10s 6ms/step - loss: 5.3867 - sparse_top_k_categorical_accuracy: 0.3846


[5.386734485626221, 0.3845940828323364]